In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from functions import *
import os
import subprocess
import requests
import re

In [2]:
enzymes = ['dl_endopeptidase', 'ld_carboxypeptidase', 
               'ld_endopeptidase', 'amidase',
               'dd_carboxypeptidase', 'diadenylate_cyclase',
               'muramidase', 'glucosaminidase']

In [ ]:
# for enzyme in enzymes:
#     df = pd.read_csv('../../data/clustering/cluster_maps/' + enzyme + 's.tsv',sep='\t')
    
#     df = df.iloc[:, 1:4]

#     df = df.dropna(subset=[df.columns[2]])

#     grouped = df.groupby(enzyme + 's-foldseek_cluster')

#     unique_counts = grouped[enzyme + 's-unclustered'].nunique()

#     single_representatives = unique_counts[unique_counts == 1]

#     foldseek_single_representatives = single_representatives.index.tolist()

#     print(f'{enzyme}: {len(foldseek_single_representatives)}')
    


In [ ]:
df = pd.read_csv('../../data/clustering/cluster_maps/dl_endopeptidases.tsv',sep='\t')

df = df.iloc[:, 1:4]

df = df[pd.isna(df[df.columns[2]])]

grouped = df.groupby('dl_endopeptidases-mmseqs_cluster')['dl_endopeptidases-unclustered'].apply(list).reset_index(name='unclustered_list')

display(grouped, len(grouped))

In [20]:
# If only 1 id in cluster, drop from df
# loop through unclustered_list, starting from second entry and download PDB
# Check if PDB is empty, if so delete and move to next entry in list, if not, move to next cluster


In [37]:
to_download = grouped['unclustered_list'].tolist()

print(to_download)

[['A0A078MK14'], ['A0A1A6AKX1'], ['A0A2U3N4J8'], ['A0A8A0RIX7'], ['A0A8D3WV90'], ['A0A8D3WY21'], ['A0A8F5H2Q7', 'A0A098EI80'], ['A0A8G2HRI5'], ['A0A8J2ZPT7'], ['A0A8S0GDS2'], ['A0A8S0GPC8'], ['A0A8S0YF99'], ['A0A916BQR8', 'A0A1V5WBM4'], ['A0A916CW89'], ['A0A916HNI5'], ['A0A916IAR6'], ['A0A916JZY9', 'W8ZK78', 'A0A919ZDK0'], ['A0A916KA10'], ['A0A916NI20'], ['A0A917ET50', 'I0JQV7'], ['A0A919WN25'], ['A0A926E6B6', 'A0A316LKK8'], ['A0A926HT18', 'J0MY21', 'A0A928K2M9', 'A0A928KYG8', 'A0A6S6XCB1', 'A0A7C7AQD7'], ['A0A927LH80'], ['A0A928LPM8', 'A0A413G4S5', 'A0A926IJ38', 'A0A498CNA2', 'A0A845RI86', 'A0A845SXI2', 'A0A3A9IJD7', 'A0A1Y4MUN7', 'A0A3C1LY33', 'A0A3B9VTI7', 'A0A6N2V297', 'A0A3D0NTZ4', 'A7VT68', 'A0A1C5YFQ8', 'A0A3A9GL57', 'A0A928K5J0', 'A0A143ZRA8', 'A0A1C5LK98', 'A0A1H8CKK5', 'A0A926I4N9', 'A0A926EA13'], ['A0A928NUY4', 'A0A928F840', 'R5PSZ8', 'A0A356GGC1'], ['A0A928PTA1', 'A0A3D5TKR5', 'A0A970U5P3'], ['A0A940NV69', 'A0A3B8NYU2'], ['A0A940ZKM3'], ['A0A942PKR4'], ['A0A949RNE6'], ['A0A

In [59]:
def download_and_check(ids, output_dir, replacements):
    replacement_found = False  # Flag to track if a replacement is found

    for id in ids:
        url = f"https://alphafold.ebi.ac.uk/files/AF-{id}-F1-model_v4.pdb"
        pdb_file_path = os.path.join(output_dir, f"AF-{id}-F1-model_v4.pdb")

        # Download the file
        response = requests.get(url)
        if response.status_code == 200:
            with open(pdb_file_path, 'wb') as file:
                file.write(response.content)

            # Check if "NoSuchKey" is in the file
            with open(pdb_file_path, 'r') as file:
                content = file.read()
                if "NoSuchKey" in content:
                    os.remove(pdb_file_path)  # Delete the file if the string is found
                else:
                    replacements.append(id)
                    replacement_found = True
                    break  # Exit the loop if the string is not found

    if not replacement_found:
        replacements.append('None')  # Append None if no replacement was found




In [40]:
# Directory containing the output files
output_dir = "C:/Users/odesa/Desktop/PDB_test/"
os.makedirs(output_dir, exist_ok=True)

# Loop over each list of IDs
replacements = []
for id_list in to_download:
    download_and_check(id_list, output_dir, replacements)
    # The function will stop processing the current list if "NoSuchKey" is not found in any file

In [41]:
print(replacements)

['A0A078MK14', 'None', 'None', 'None', 'None', 'None', 'A0A098EI80', 'None', 'None', 'None', 'None', 'None', 'A0A1V5WBM4', 'None', 'None', 'None', 'W8ZK78', 'None', 'None', 'I0JQV7', 'None', 'A0A316LKK8', 'J0MY21', 'None', 'A0A413G4S5', 'R5PSZ8', 'A0A3D5TKR5', 'A0A3B8NYU2', 'None', 'None', 'None', 'A0A357WRL2', 'None', 'A0A1Y2MSQ6', 'R6YTV6', 'None', 'A0A6A8FPK7']


In [ ]:
grouped['unclustered_list'] = replacements
print(grouped)

In [51]:
original = pd.read_csv('../../data/clustering/cluster_maps/dl_endopeptidases.tsv',sep='\t')

df = df.iloc[:, 1:4]

In [52]:
merged_df = pd.merge(original, grouped, on='dl_endopeptidases-mmseqs_cluster', how='left')

original['replacements'] = merged_df['unclustered_list']

In [55]:
original['dl_endopeptidases-mmseqs_cluster'] = np.where(original['replacements'].notna(), 
                                                        original['replacements'], original['dl_endopeptidases-mmseqs_cluster'])

In [58]:
# Filter the dataframe based on the condition
filtered_df = grouped[grouped['unclustered_list'] == 'None']

# Get the values from the 'dl_endopeptidases-mmseqs_cluster' column
values = filtered_df['dl_endopeptidases-mmseqs_cluster'].values

# Create a text file and write the values to it
with open('E:/PDBs/dl_endopeptidases_representatives/to_delete.txt', 'w') as file:
    for value in values:
        file.write(str(value) + '\n')


In [67]:
# Needs to be made into a function and optimized. Shoul dimplement multiprocessing for
# downloading the PDBs and checking if they are empty


enzymes = ['dl_endopeptidase', 'ld_carboxypeptidase', 
               'ld_endopeptidase', 'amidase',
               'dd_carboxypeptidase', 'diadenylate_cyclase',
               'muramidase', 'glucosaminidase']

for enzyme in enzymes:
    
    # read in the cluster map, drop the first column
    df = pd.read_csv('../../data/clustering/cluster_maps/' + enzyme + 's.tsv',sep='\t')
    df = df.iloc[:, 1:4]

    # drop the rows where the foldseek cluster is not empty
    no_foldseek = df[pd.isna(df[df.columns[2]])]

    # group by the mmseqs cluster and create a list of the unclustered ids
    grouped = no_foldseek.groupby(f'{enzyme}s-mmseqs_cluster')[f'{enzyme}s-unclustered'].apply(list).reset_index(name='unclustered_list')

    # create a directory for the PDB files
    to_download = grouped['unclustered_list'].tolist()
    output_dir = f"C:/Users/odesa/Desktop/PDB_test/{enzyme}"
    os.makedirs(output_dir, exist_ok=True)

    # Loop over each list of IDs
    replacements = []
    for id_list in to_download:
        # download new PDBs, check if empty, creating a list of replacements
        download_and_check(id_list, output_dir, replacements)

    # Replace the 'unclustered_list' column with the list of replacements
    grouped['unclustered_list'] = replacements

    # Merge the original dataframe with the new one on mmseqs cluster
    merged_df = pd.merge(df, grouped, on=f'{enzyme}s-mmseqs_cluster', how='left')

    # Create a new column with the list of replacements
    df['replacements'] = merged_df['unclustered_list']

    # Replace the mmseqs cluster with the list of replacements if it is not empty
    df[f'{enzyme}s-mmseqs_cluster'] = np.where(df['replacements'].notna(), 
                                                df['replacements'], 
                                                df[f'{enzyme}s-mmseqs_cluster'])

    # Filter the dataframe based on the condition
    filtered_df = grouped[grouped['unclustered_list'] == 'None']

    # Get the values from the 'dl_endopeptidases-mmseqs_cluster' column
    values = filtered_df[f'{enzyme}s-mmseqs_cluster'].values 

    # Create a text file and write the values to it
    with open(f'C:/Users/odesa/Desktop/PDB_test/{enzyme}/to_delete.txt', 'w') as file:
        for value in values:
            file.write(str(value) + '\n')

    df.to_csv(f'C:/Users/odesa/Desktop/{enzyme}_cluster_update.tsv', sep='\t', index=False)

In [85]:

for enzyme in enzymes:
    df = pd.read_csv(f'C:/Users/odesa/Desktop/{enzyme}_cluster_update.tsv', sep='\t')

    # df[f'{enzyme}s-mmseqs_cluster'] = df.apply(lambda row: row[f'{enzyme}s-unclustered'] if pd.isna(row[f'{enzyme}s-mmseqs_cluster']) else row[f'{enzyme}s-mmseqs_cluster'], axis=1)
    
    # df.to_csv(f'C:/Users/odesa/Desktop/{enzyme}_cluster_update.tsv', sep='\t', index=False)
    
    unique_values = df[f'{enzyme}s-mmseqs_cluster'].unique().tolist()

    downloaded = []
    with open(f'E:/PDBs/{enzyme}s_representatives/all_files.txt', 'r') as file:
        for line in file:
            downloaded.append(line.strip())

    to_remove = [x for x in downloaded if x not in unique_values]
    print(f'{enzyme}: {len(to_remove)} {to_remove}')

    with open(f'E:/PDBs/{enzyme}s_representatives/to_remove.txt', 'w') as file:
        for value in to_remove:
            file.write(str(value) + '\n')







dl_endopeptidase: 15 ['A0A3D0XRE2', 'A0A8F5H2Q7', 'A0A916BQR8', 'A0A916JZY9', 'A0A917ET50', 'A0A926E6B6', 'A0A926HT18', 'A0A928LPM8', 'A0A928NUY4', 'A0A928PTA1', 'A0A940NV69', 'A0A970N7P1', 'A0A977II70', 'R7ITK9', 'UPI001BDB9821']
ld_carboxypeptidase: 335 ['A0A1Z8Z7X7', 'A0A2D5G1M9', 'A0A2D9HD30', 'A0A2E2QCN2', 'A0A2E4LMV6', 'A0A2E4MYF1', 'A0A3C1YV70', 'A0A8E3N132', 'A0A8G1NXZ2', 'A0A8I1LD88', 'A0A8I2AY56', 'A0A8I2D7L2', 'A0A8J3EKZ4', 'A0A8J6TF70', 'A0A8J7US32', 'A0A8J8MJA0', 'A0A8T3MTU4', 'A0A8T3SGW3', 'A0A8T6GUD8', 'A0A8T9E7Z9', 'A0A915TUM5', 'A0A921KZ64', 'A0A922ZFA2', 'A0A923H7T4', 'A0A923VBU8', 'A0A924LY46', 'A0A924UL56', 'A0A924WSA9', 'A0A925G1K5', 'A0A925JLY3', 'A0A925PTW1', 'A0A925YE95', 'A0A926CFG4', 'A0A927MT85', 'A0A927PF25', 'A0A927X2C9', 'A0A928B8P7', 'A0A928UF28', 'A0A928VBX5', 'A0A929JI43', 'A0A929JNP9', 'A0A929ZIM8', 'A0A930A3G7', 'A0A930A8L3', 'A0A930E7Z9', 'A0A931DRW6', 'A0A932CNA5', 'A0A932DWM8', 'A0A932EFG7', 'A0A933ZBJ6', 'A0A934FXG6', 'A0A935BPW1', 'A0A935I3Y3', '